# 1d Convolutional Neural Networks

In [0]:

# Imports
import numpy as np
import os
# from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import tensorflow as tf
from tensorflow import keras

### About the Data

Data is from people doing one of these activities (WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING) <br><br>
The data is from a smartphone and provides the body acceleration, general acceleration and a gyroscope reading <br><br>
Data is from: https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones 

### Read in data from functions provided below

In [0]:
X_train, labels_train, list_ch_train = read_data_train() # train
X_test, labels_test, list_ch_test = read_data_test()

In [9]:
print(X_train.shape)

(7352, 128, 9)



#### Explaining the shape of the data 
There are 7,352 data points that are classified as doing one of those activities <br>
There are 128 time steps <br>
There are 9 values for each time step for each data point for the x,y,z values of the body 
acceleration, general acceleration and gyrscope reading. <br>

### Use train_test_split to create training and validation data and labels

In [0]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, labels_train, stratify = labels_train)

In [12]:
print(X_tr.shape)

(5514, 128, 9)


One hot coding is when each row represents one label and it the nonzero column represents what the label is<br>
 [0,0,0,0,0,1] <br>
           ^ This label is LAYING, which was previously represented with a 5
           <br><br>

### One hot encoding functions are provided at the bottom of this notebook <br> Use these functions to convert the training, validation and testing labels

In [0]:
y_tr = one_hot(y_tr)
y_val = one_hot(y_val)
y_test = one_hot(labels_test)

### Define your Keras model <br>
You wlil want to use

See for examples on creating Keras modelshttps://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

In [19]:
# fill in with your code below
model = keras.Sequential()
model.add(keras.layers.Conv1D(filters = 18, 
                 kernel_size=3,
                 strides = 1,
                 padding = 'same',
                 activation='relu',
                 input_shape=X_tr[0].shape) )
model.add(keras.layers.MaxPooling1D(pool_size=2, padding = 'valid'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(6, activation='sigmoid'))
print(model.summary())  # view model
    
    


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 128, 18)           504       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 64, 18)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 6918      
Total params: 7,422
Trainable params: 7,422
Non-trainable params: 0
_________________________________________________________________
None


### Compile your model

In [0]:
lr = 0.008 # choose a learning rate, this will be a good paramter to tune
model.compile( optimizer=tf.train.AdamOptimizer(lr), loss=keras.losses.categorical_crossentropy, metrics = ['accuracy'] )

### Fit your model

In [21]:
history = model.fit( X_tr,
                    y_tr,
                    epochs= 10,
                    batch_size= 20,
                    validation_data=(X_val, y_val),
                    verbose=1 
                    )

Train on 5514 samples, validate on 1838 samples
Epoch 1/10
5514/5514 [==============================] - 1s 233us/step - loss: 0.4823 - acc: 0.7913 - val_loss: 0.2125 - val_acc: 0.9064
Epoch 2/10
5514/5514 [==============================] - 1s 184us/step - loss: 0.2126 - acc: 0.9084 - val_loss: 0.2435 - val_acc: 0.9081
Epoch 3/10
5514/5514 [==============================] - 1s 170us/step - loss: 0.1669 - acc: 0.9224 - val_loss: 0.1961 - val_acc: 0.9102
Epoch 4/10
5514/5514 [==============================] - 1s 166us/step - loss: 0.1377 - acc: 0.9354 - val_loss: 0.1566 - val_acc: 0.9293
Epoch 5/10
5514/5514 [==============================] - 1s 175us/step - loss: 0.1542 - acc: 0.9296 - val_loss: 0.1716 - val_acc: 0.9287
Epoch 6/10
5514/5514 [==============================] - 1s 173us/step - loss: 0.1292 - acc: 0.9373 - val_loss: 0.1444 - val_acc: 0.9287
Epoch 7/10
5514/5514 [==============================] - 1s 175us/step - loss: 0.1383 - acc: 0.9336 - val_loss: 0.1335 - val_acc: 0.9374


5514/5514 [==============================] - 1s 178us/step - loss: 0.1219 - acc: 0.9412 - val_loss: 0.1627 - val_acc: 0.9325
Epoch 10/10
5514/5514 [==============================] - 1s 172us/step - loss: 0.1645 - acc: 0.9305 - val_loss: 0.1758 - val_acc: 0.9293


### Code below prints the test loss and accuracy <br>
### Before you print your accuracy and loss, play around with the parameters!!

In [18]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

('Test loss:', 0.46549361224002744)
('Test accuracy:', 0.8873430607397353)


### Provided Functions Below

In [0]:
def read_data_train():
  """ Read data """

  # Fixed params
  n_class = 6
  n_steps = 128

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/y_train.txt'
  labels = pd.read_csv(label_path, header = None)

  list_of_channels = ['body_acc_x', 'body_acc_y', 'body_acc_z', 'body_gyro_x',
  'body_gyro_y', 'body_gyro_z', 'total_acc_x', 'total_acc_y', 'total_acc_z']

  X = np.zeros((len(labels), n_steps, len(list_of_channels)))

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/body_acc_x_train.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,0] = dat_.as_matrix()

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/body_acc_y_train.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,1] = dat_.as_matrix()

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/body_acc_z_train.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,2] = dat_.as_matrix()

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/body_gyro_x_train.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,3] = dat_.as_matrix()

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/body_gyro_y_train.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,4] = dat_.as_matrix()

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/body_gyro_z_train.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,5] = dat_.as_matrix()

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/total_acc_x_train.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,6] = dat_.as_matrix()

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/total_acc_y_train.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,7] = dat_.as_matrix()

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/total_acc_z_train.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,8] = dat_.as_matrix()


  # Return 
  return X, labels[0].values, list_of_channels

In [0]:
def read_data_test():
  """ Read data """

  # Fixed params
  n_class = 6
  n_steps = 128

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/y_test.txt'
  labels = pd.read_csv(label_path, header = None)

  list_of_channels = ['body_acc_x', 'body_acc_y', 'body_acc_z', 'body_gyro_x',
  'body_gyro_y', 'body_gyro_z', 'total_acc_x', 'total_acc_y', 'total_acc_z']

  X = np.zeros((len(labels), n_steps, len(list_of_channels)))

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/body_acc_x_test.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,0] = dat_.as_matrix()

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/body_acc_y_test.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,1] = dat_.as_matrix()

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/body_acc_z_test.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,2] = dat_.as_matrix()

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/body_gyro_x_test.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,3] = dat_.as_matrix()

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/body_gyro_y_test.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,4] = dat_.as_matrix()

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/body_gyro_z_test.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,5] = dat_.as_matrix()

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/total_acc_x_test.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,6] = dat_.as_matrix()

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/total_acc_y_test.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,7] = dat_.as_matrix()

  label_path = 'https://raw.githubusercontent.com/BeaverWorksMedlytics/datasets/master/week2_conv1d/IntertialSignals/total_acc_z_test.txt'
  dat_ = pd.read_csv(label_path, delim_whitespace = True, header = None)
  X[:,:,8] = dat_.as_matrix()


  # Return 
  return X, labels[0].values, list_of_channels

In [0]:
def one_hot(labels, n_class = 6):
	""" One-hot encoding """
	expansion = np.eye(n_class)
	y = expansion[:, labels-1].T
	assert y.shape[1] == n_class, "Wrong number of labels!"

	return y

In [0]:
def get_batches(X, y, batch_size = 100):
	""" Return a generator for batches """
	n_batches = len(X) // batch_size
	X, y = X[:n_batches*batch_size], y[:n_batches*batch_size]

	# Loop over batches and yield
	for b in range(0, len(X), batch_size):
		yield X[b:b+batch_size], y[b:b+batch_size]